In [9]:
# All necessary libraries must be put here
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf
import plotly.graph_objects as go
import pandas as pd
import cv2
import matplotlib.image as mpimg
import os
import warnings

In [2]:
# Read the CRSP Data and make it become a dataframe
crsp_data_1993_2019 = pd.read_csv("/Users/namnguyen/Desktop/Research/Don/Imaging_Price_Trends_Code/crsp_data_20240719_1.csv")

# Check the data
crsp_data_1993_2019.head(5)

/var/folders/pl/z1lfm5xs7299k23shpwd6ngh0000gp/T/ipykernel_14451/4244800754.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp_data_1993_2019 = pd.read_csv("/Users/namnguyen/Desktop/Research/Don/Imaging_Price_Trends_Code/crsp_data_20240719_1.csv")


,PERMNO,HdrCUSIP,Ticker,PERMCO,DlyCalDt,DlyVol,DlyClose,DlyLow,DlyHigh,DlyOpen
0,10001,36720410,GFGC,7953,1993-01-04,150.0,14.5,14.50,14.5,14.50
1,10001,36720410,GFGC,7953,1993-01-05,0.0,NaN,NaN,NaN,NaN
2,10001,36720410,GFGC,7953,1993-01-06,0.0,NaN,NaN,NaN,NaN
3,10001,36720410,GFGC,7953,1993-01-07,228.0,14.5,14.50,14.5,14.50
4,10001,36720410,GFGC,7953,1993-01-08,1375.0,14.5,14.25,14.5,14.25


In [4]:
# At this part, we focus first on training sample
# Training data are taken from 1993 to 2000. 

crsp_data_1993_2019["DlyCalDt"] = pd.to_datetime(crsp_data_1993_2019["DlyCalDt"])

# Filter dates from 1993 to 2000
start_date = "1993-01-01"
end_date = "2000-12-31" 
crsp_data_1993_2000 = crsp_data_1993_2019[(crsp_data_1993_2019["DlyCalDt"] >= start_date) & (crsp_data_1993_2019["DlyCalDt"] <= end_date)]

# Check the last 5 values
print(crsp_data_1993_2000.tail(5))

# Check the shape of the crsp_data_1993_2000
print(crsp_data_1993_2000.shape) # (17223740, 10)

# Check the shape of the crsp_data_1993_2019
print(crsp_data_1993_2019.shape) # (51131155, 10)

          PERMNO  HdrCUSIP Ticker  PERMCO   DlyCalDt   DlyVol  DlyClose  \
50923819   93316  98960110   ZING    7469 2000-03-13  10685.0   15.3750   
50923820   93316  98960110   ZING    7469 2000-03-14   1800.0   15.0625   
50923821   93316  98960110   ZING    7469 2000-03-15   2100.0   15.0625   
50923822   93316  98960110   ZING    7469 2000-03-16    700.0   15.3750   
50923823   93316  98960110    NaN    7469 2000-03-17      NaN       NaN   

           DlyLow  DlyHigh  DlyOpen  
50923819  15.1250   15.375  15.2500  
50923820  15.0625   15.375  15.3750  
50923821  15.0625   15.375  15.0625  
50923822  15.0625   15.375  15.0625  
50923823      NaN      NaN      NaN  
(17223740, 10)
(51131155, 10)


In [5]:
# Replace the NaN values with 0
crsp_data_1993_2000 = crsp_data_1993_2000.dropna()

# Check the shape again
print(crsp_data_1993_2000.shape)

(15555109, 10)


In [6]:
# Drop unused columns in the dataframe
crsp_data_1993_2000 = crsp_data_1993_2000.drop(["HdrCUSIP", "Ticker", "PERMCO"], axis = 1)

# Check the shape again
print(crsp_data_1993_2000.shape) # (15555109, 7)

(15555109, 7)


In [7]:
# Function to calculate 5-day log returns
def calculate_5_day_log_returns(group):
    # Create return for each day
    group["Return"] = group["DlyClose"].pct_change()

    # Shift it back 1 column
    group["Return"] = group["Return"].shift(-1)
    group["Return"].fillna(method = "ffill", inplace = True)
    
    # Calculate Log Returns
    group["Log_Return"] = np.log(group["Return"] + 1) # remember that x = ln(1 + x), Taylor's polynomial on e^x

    # Forward 5-day return
    group["5_day_return"] = group["Log_Return"].rolling(5).sum().shift(-5)
    return group 

In [10]:
# Suppress FutureWarnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

# Calculate 5-day stock returns for the whole dataset from 1993 to 2000
crsp_data_1993_2000_R5 = crsp_data_1993_2000.groupby("PERMNO").apply(calculate_5_day_log_returns)

# Reset the index
crsp_data_1993_2000_R5 = crsp_data_1993_2000_R5.reset_index(drop = True)

# Display the result
crsp_data_1993_2000_R5

/var/folders/pl/z1lfm5xs7299k23shpwd6ngh0000gp/T/ipykernel_14451/3732077754.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  crsp_data_1993_2000_R5 = crsp_data_1993_2000.groupby("PERMNO").apply(calculate_5_day_log_returns)


,PERMNO,DlyCalDt,DlyVol,DlyClose,DlyLow,DlyHigh,DlyOpen,Return,Log_Return,5_day_return
0,10001,1993-01-04,150.0,14.5000,14.5000,14.500,14.5000,0.000000,0.000000,-5.310983e-02
1,10001,1993-01-07,228.0,14.5000,14.5000,14.500,14.5000,0.000000,0.000000,-1.739174e-02
2,10001,1993-01-08,1375.0,14.5000,14.2500,14.500,14.2500,0.000000,0.000000,-1.739174e-02
3,10001,1993-01-11,200.0,14.5000,14.5000,14.500,14.5000,-0.051724,-0.053110,1.387779e-17
4,10001,1993-01-13,700.0,13.7500,13.7500,14.000,14.0000,0.000000,0.000000,1.387779e-17
...,...,...,...,...,...,...,...,...,...,...
15555104,93316,2000-03-10,8555.0,15.2500,15.1250,15.250,15.1250,0.008197,0.008163,NaN
15555105,93316,2000-03-13,10685.0,15.3750,15.1250,15.375,15.2500,-0.020325,-0.020535,NaN
15555106,93316,2000-03-14,1800.0,15.0625,15.0625,15.375,15.3750,0.000000,0.000000,NaN
15555107,93316,2000-03-15,2100.0,15.0625,15.0625,15.375,15.0625,0.020747,0.020535,NaN
